### nbconvert
Jupyterlab utilizes the nbconvert library to perform transformations on .ipynb files. The options from ```File -> Export Notebook As... -> *``` are available via nbconvert. 

They provide a built in option to export files as scripts which is the manual process I have been doing: ```File -> Export Notebook As... -> Executable Script ```

This is also callable via CLI as: ```jupyter nbconvert  --to script WHILE_interpreter.ipynb ```

Under the hood, nbconvert is taking a notebook file and applying an Exporter() to it. The implementation of the 'script' Exporter can be found here and is meant to be extended to implement custom Exporters: https://github.com/jupyter/nbconvert/blob/master/nbconvert/exporters/script.py

``` """Generic script exporter class for any kernel language"""

# Copyright (c) Jupyter Development Team.
# Distributed under the terms of the Modified BSD License.

import entrypoints
from .templateexporter import TemplateExporter

from traitlets import Dict, default
from .base import get_exporter


class ScriptExporter(TemplateExporter):
    # Caches of already looked-up and instantiated exporters for delegation:
    _exporters = Dict()
    _lang_exporters = Dict()
    export_from_notebook = "Script"

    @default('template_file')
    def _template_file_default(self):
        return 'script.j2'

    @default('template_name')
    def _template_name_default(self):
        return 'script'

    def _get_language_exporter(self, lang_name):
        """Find an exporter for the language name from notebook metadata.
        Uses the nbconvert.exporters.script group of entry points.
        Returns None if no exporter is found.
        """
        if lang_name not in self._lang_exporters:
            try:
                Exporter = entrypoints.get_single(
                    'nbconvert.exporters.script', lang_name).load()
            except entrypoints.NoSuchEntryPoint:
                self._lang_exporters[lang_name] = None
            else:
                # TODO: passing config is wrong, but changing this revealed more complicated issues
                self._lang_exporters[lang_name] = Exporter(config=self.config, parent=self)
        return self._lang_exporters[lang_name]

    def from_notebook_node(self, nb, resources=None, **kw):
        langinfo = nb.metadata.get('language_info', {})

        # delegate to custom exporter, if specified
        exporter_name = langinfo.get('nbconvert_exporter')
        if exporter_name and exporter_name != 'script':
            self.log.debug("Loading script exporter: %s", exporter_name)
            if exporter_name not in self._exporters:
                Exporter = get_exporter(exporter_name)
                # TODO: passing config is wrong, but changing this revealed more complicated issues
                self._exporters[exporter_name] = Exporter(config=self.config, parent=self)
            exporter = self._exporters[exporter_name]
            return exporter.from_notebook_node(nb, resources, **kw)

        # Look up a script exporter for this notebook's language
        lang_name = langinfo.get('name')
        if lang_name:
            self.log.debug("Using script exporter for language: %s", lang_name)
            exporter = self._get_language_exporter(lang_name)
            if exporter is not None:
                return exporter.from_notebook_node(nb, resources, **kw)

        # Fall back to plain script export
        self.file_extension = langinfo.get('file_extension', '.txt')
        self.output_mimetype = langinfo.get('mimetype', 'text/plain')
        return super().from_notebook_node(nb, resources, **kw) 
```

Examples on how to extend a built in exporter are here: https://nbconvert.readthedocs.io/en/latest/external_exporters.html#external-exporters

Below I adapt their code write my own.

In [ ]:
# file __init__.py
import os
import os.path

from traitlets.config import Config
from nbconvert.exporters.script import ScriptExporter

In [ ]:
#-----------------------------------------------------------------------------
# Classes
#-----------------------------------------------------------------------------

class ScalaExporter(ScriptExporter):
    """
    My custom exporter
    """

    # If this custom exporter should add an entry to the
    # "File -> Download as" menu in the notebook, give it a name here in the
    # `export_from_notebook` class member
    export_from_notebook = ".sc (Scala Script)"

    def _file_extension_default(self):
        """
        The new file extension is ``.sc``
        """
        return '.sc'

#     @property
#     def template_paths(self):
#         """
#         We want to inherit from HTML template, and have template under
#         ``./templates/`` so append it to the search path. (see next section)

#         Note: nbconvert 6.0 changed ``template_path`` to ``template_paths``
#         """
#         return super().template_paths+[os.path.join(os.path.dirname(__file__), "templates")]

#     def _template_file_default(self):
#         """
#         We want to use the new template we ship with our library.
#         """
#         return 'test_template' # full

### Running the script:
```jupyter nbconvert  --to export_scala.ScalaExport notebook.ipynb ```


In [ ]:
class A:
    def __init__(self):
       pass
    def add(self, a, b):
        return a + b

class B(A): # this add overrides parents
    def __init__(self):
       pass
    def add(self, a, b):
        return a - b

x = A()
x.add(4, 3)

y = B()
y.add(4, 3)
